In [1]:
import sqlite3
import datetime

DB_NAME = "personen.db"

# --- Datenbank einrichten ---
def init_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS personen (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            vorname TEXT NOT NULL,
            nachname TEXT NOT NULL,
            geburtsdatum TEXT NOT NULL,
            anschrift TEXT NOT NULL,
            koerpergroesse INTEGER NOT NULL,
            augenfarbe TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

# --- Eingabevalidierungen ---
def validiere_datum(datum_str):
    try:
        geburtsdatum = datetime.datetime.strptime(datum_str, "%d.%m.%Y").date()
        if geburtsdatum > datetime.date.today():
            print("❌ Fehler: Geburtsdatum darf nicht in der Zukunft liegen.")
            return None
        return geburtsdatum
    except ValueError:
        print("❌ Fehler: Bitte Datum im Format TT.MM.JJJJ eingeben.")
        return None

def validiere_koerpergroesse(groesse_str):
    if not groesse_str.isdigit():
        print("❌ Fehler: Körpergröße muss eine Zahl sein.")
        return None
    groesse = int(groesse_str)
    if groesse <= 0:
        print("❌ Fehler: Körpergröße muss positiv sein.")
        return None
    return groesse

# --- Daten hinzufügen ---
def neue_person():
    vorname = input("Vorname: ").strip()
    nachname = input("Nachname: ").strip()
    anschrift = input("Anschrift: ").strip()
    augenfarbe = input("Augenfarbe: ").strip()

    # Geburtsdatum validieren
    while True:
        geburtsdatum_str = input("Geburtsdatum (TT.MM.JJJJ): ").strip()
        geburtsdatum = validiere_datum(geburtsdatum_str)
        if geburtsdatum:
            break

    # Körpergröße validieren
    while True:
        groesse_str = input("Körpergröße (cm): ").strip()
        groesse = validiere_koerpergroesse(groesse_str)
        if groesse:
            break

    if not all([vorname, nachname, anschrift, augenfarbe]):
        print("❌ Fehler: Alle Felder müssen ausgefüllt sein.")
        return

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO personen (vorname, nachname, geburtsdatum, anschrift, koerpergroesse, augenfarbe)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (vorname, nachname, geburtsdatum.strftime("%d.%m.%Y"), anschrift, groesse, augenfarbe))
    conn.commit()
    conn.close()
    print("✅ Person erfolgreich hinzugefügt!\n")

# --- Alle Personen anzeigen ---
def alle_personen_anzeigen():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM personen")
    personen = cursor.fetchall()
    conn.close()

    if not personen:
        print("Keine Datensätze vorhanden.\n")
        return

    print("\n--- Personenliste ---")
    print("{:<5} {:<15} {:<15} {:<12} {:<25} {:<12} {:<10}".format(
        "ID", "Vorname", "Nachname", "Geburtsdatum", "Anschrift", "Größe", "Augenfarbe"
    ))
    print("-" * 95)
    for p in personen:
        print("{:<5} {:<15} {:<15} {:<12} {:<25} {:<12} {:<10}".format(*p))
    print()

# --- Menü ---
def menue():
    while True:
        print("=== Personaldatenverwaltung ===")
        print("1 – Neue Person hinzufügen")
        print("2 – Alle Personen anzeigen")
        print("3 – Beenden")
        wahl = input("Bitte wählen: ").strip()

        if wahl == "1":
            neue_person()
        elif wahl == "2":
            alle_personen_anzeigen()
        elif wahl == "3":
            print("Programm beendet.")
            break
        else:
            print("❌ Ungültige Auswahl.\n")

# --- Start ---
if __name__ == "__main__":
    init_db()
    menue()


=== Personaldatenverwaltung ===
1 – Neue Person hinzufügen
2 – Alle Personen anzeigen
3 – Beenden
Keine Datensätze vorhanden.

=== Personaldatenverwaltung ===
1 – Neue Person hinzufügen
2 – Alle Personen anzeigen
3 – Beenden
✅ Person erfolgreich hinzugefügt!

=== Personaldatenverwaltung ===
1 – Neue Person hinzufügen
2 – Alle Personen anzeigen
3 – Beenden

--- Personenliste ---
ID    Vorname         Nachname        Geburtsdatum Anschrift                 Größe        Augenfarbe
-----------------------------------------------------------------------------------------------
1     sascha          schilling       02.06.1995   an der turnhalle          200          braun     

=== Personaldatenverwaltung ===
1 – Neue Person hinzufügen
2 – Alle Personen anzeigen
3 – Beenden
Programm beendet.
